### ORI Final Analysis
### 01. Constraints

#### 10/15/2024
#### This script processes data and creates the constraints submodel. 

In [2]:
#import system modules and set environment
import arcpy
import os
from arcpy import env
from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI'

#define spatial reference (EPSG for NAD 83 UTM Zone 11N)
spatial_ref = arcpy.SpatialReference(26911)

#create a copy of the ORI area hex grid
hex_grid = 'data/ORI_Area_of_Interest_grid.shp'
constraints_submodel = 'data/input_data/constraints_submodel'
arcpy.management.CopyFeatures(hex_grid, constraints_submodel)

#define area of interest
aoi = 'data/ORI_Area_of_Interest.shp'

#create geopackage
cs_gpkg_path = 'data/input_data/ORI_Constraints.gpkg'
if not os.path.exists(cs_gpkg_path):
    arcpy.management.CreateSQLiteDatabase(cs_gpkg_path, "GEOPACKAGE")

In [3]:
# Wrecks and Obstructions (ori_cs_01)

#define file
gpkg_path = 'data/source_data/WreckObstruction.gpkg'
file_path = f"{gpkg_path}/main.WreckObstruction"

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Feet")

#save to geopackage
output_name = 'Wreck_Obs_500ft'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_01', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_01', '1', field_type="DOUBLE")


<Result 'constraints_submodel_Layer2'>

In [4]:
# CalCOFI Sites (ori_cs_02)

#define file
file_path = 'data/source_data/CalCOFI_Survey.shp'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Meters")

#save to geopackage
output_name = 'CalCOFI_500m'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_02', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_02', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer4'>

In [5]:
# Ferry Routes (ori_cs_03)

#define file
file_path = 'data/source_data/Ferry_Routes.shp'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Meters")

#save to geopackage
output_name = 'Ferry_Routes_500m'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_03', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_03', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer6'>

In [6]:
# Hardbottom (ori_cs_04)

#define file
file_url = 'https://tiledimageservices2.arcgis.com/Uq9r85Potqm3MfRV/arcgis/rest/services/biosds3091_cru/ImageServer'
file_path = 'in_memory/file_path'
arcpy.management.MakeImageServerLayer(file_url, file_path)

#extract by mask using buffered AOI
aoi_buff = 'in_memory/aoi_buff'
arcpy.analysis.Buffer(aoi, aoi_buff, "1000 Meters")

raster_path = 'in_memory/raster_path'
out_raster = arcpy.sa.ExtractByMask(file_path, aoi_buff, "INSIDE")
out_raster.save(raster_path)

#convert raster to polgon
poly_path = 'in_memory/poly_path'
arcpy.conversion.RasterToPolygon(out_raster, poly_path, "SIMPLIFY", "CLASSNAME","SINGLE_OUTER_PART")

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(poly_path, file_project, spatial_ref)

#select only harbottom
harbottom = arcpy.management.SelectLayerByAttribute(file_project, "NEW_SELECTION", where_clause = "CLASSNAME = 'Hard'")
                                                    
#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(harbottom, file_buff, "500 Feet")

#save to geopackage
output_name = 'Harbottom_500ft'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_04', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_04', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer8'>

In [7]:
# JOFLO Corridors (ori_cs_05)

#define file
file_path = 'data/source_data/AOA_Industry.gdb/SoCal_JOFLO_Cooridors'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Feet")

#save to geopackage
output_name = 'JOFLO_500ft'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_05', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_05', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer10'>

In [8]:
# Navigable Waterways (ori_cs_06)

#define file
file_path = 'data/source_data/CNW_V6_2024.shp'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Meters")

#save to geopackage
output_name = 'Navigable_Water_500m'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_06', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_06', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer12'>

In [9]:
# Oil and Gas Pipelines (ori_cs_07)

#define file
file_path = 'data/source_data/Pipeline_Areas.shp'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Meters")

#save to geopackage
output_name = 'OG_Pipelines_500m'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_07', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_07', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer14'>

In [10]:
# Oil and Gas Wells (ori_cs_08)

#define file
file_path = 'data/source_data/OffshoreOilGasWell.gpkg/main.OffshoreOilGasWell'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Meters")

#save to geopackage
output_name = 'OG_Wells_500m'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_08', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_08', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer16'>

In [11]:
# Oil and Gas Platforms (ori_cs_09)

#define file
file_path = 'data/source_data/OffshoreOilGasPlatform.gpkg/main.OffshoreOilGasPlatform'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Meters")

#save to geopackage
output_name = 'OG_Platforms_500m'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_09', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_09', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer18'>

In [12]:
# Submarine Cables (ori_cs_10)

#define file
file_path = 'data/source_data/SubmarineCable.gpkg/main.SubmarineCable'

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project(file_path, file_project, spatial_ref)

#buffer
file_buff = 'in_memory/file_buff'
arcpy.analysis.Buffer(file_project, file_buff, "500 Meters")

#save to geopackage
output_name = 'Sub_Cables_500m'
arcpy.conversion.FeatureClassToFeatureClass(file_buff, cs_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel,
                                                                     'INTERSECT',
                                                                     file_buff)
arcpy.management.CalculateField(file_selection, 'ori_cs_10', '0', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(constraints_submodel, 
                                                        'INTERSECT', 
                                                        file_buff, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_cs_10', '1', field_type="DOUBLE")

<Result 'constraints_submodel_Layer20'>

In [13]:
# Constraints Submodel

#call shapefile
constraints_submodel = 'data/input_data/constraints_submodel.shp'

#create a new field for constraints
constraint_field = "cs_final"

existing_fields = [f.name for f in arcpy.ListFields(constraints_submodel)]

if constraint_field not in existing_fields:
    arcpy.management.AddField(constraints_submodel, constraint_field, 'DOUBLE')
    
#expression to get final constraints
final_expression = '(!ori_cs_01! * !ori_cs_02! * !ori_cs_03! * !ori_cs_04! * !ori_cs_05! * !ori_cs_06! * !ori_cs_07! * !ori_cs_08! * !ori_cs_09! * !ori_cs_10!)'


#calculate the geometric mean and store the result in the 'final_cs' attribute
arcpy.CalculateField_management(constraints_submodel, constraint_field, final_expression, 'PYTHON3')


<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data/constraints_submodel.shp'>

In [14]:
# Export constrained hexes

#create feature layer for the file
arcpy.management.MakeFeatureLayer(constraints_submodel, "constrained")

#select file where it is constrained
arcpy.management.SelectLayerByAttribute("constrained", "NEW_SELECTION", '"cs_final" <> 0')

#export the selection
out_path = 'data/input_data'
out_name = 'constrained.shp'

arcpy.conversion.FeatureClassToFeatureClass("constrained", out_path, out_name)


<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data\\constrained.shp'>